#Get the Data

The reference, source, and hypothesis file are here:
[wmt-conference/wmt24-news-systems/tree/main/txt](https://github.com/wmt-conference/wmt24-news-systems/tree/main/txt)

In [ ]:
!git clone https://github.com/wmt-conference/wmt24-news-systems.git

Cloning into 'wmt24-news-systems'...
remote: Enumerating objects: 711, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (84/84), done.
remote: Total 711 (delta 56), reused 83 (delta 28), pack-reused 595 (from 1)
Receiving objects: 100% (711/711), 194.68 MiB | 15.31 MiB/s, done.
Resolving deltas: 100% (317/317), done.
Updating files: 100% (596/596), done.


#Evaluate with MetricX

In [ ]:
!pip install datasets accelerate
!pip install git+https://github.com/google-research/mt-metrics-eval
!git clone https://github.com/google-research/metricx.git
!mv metricx/metricx23/predict.py metricx/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Cloning https://github.com/google-research/mt-metrics-eval t

Preparation of the dataset:
* **/!\ This code was used before we introduced the canary string in the files. You have to remove the first line of the hypothesis and reference files before running Metricx. /!\**

In [ ]:
!mkdir -p ./MetricX/json
#The translation directions to evaluate
translation_directions=["cs-uk", "en-cs", "en-de", "en-is", "en-es", "en-ja", "en-ru", "en-uk", "en-hi", "en-zh", "ja-zh"]
data_dir = "/content/wmt24-news-systems/txt/"

In [ ]:
import os,json
def make_metricX_json(tdirection):
  tdirection_dir = "/content/MetricX/json/"+tdirection
  os.system("mkdir "+tdirection_dir)
  reference = open(data_dir+"/references/{tdirection}.refA.txt".format(tdirection=tdirection)).readlines()
  hypothesis_directory = data_dir+"/system-outputs/"+tdirection
  hypothesis_files = [os.path.join(hypothesis_directory, f) for f in os.listdir(hypothesis_directory) if f.endswith(".txt") and os.path.isfile(os.path.join(hypothesis_directory, f))]
  system_names = set()
  for hfile in hypothesis_files:
      system_name = "".join(hfile.split('/')[-1].split('.')[:-1])
      system_names.add(system_name)
      print(system_name)
      json_file = tdirection_dir+"/"+system_name+".jsonl"
      hf = open(hfile).readlines()
      json_lines = []
      for l in range(len(hf)):
          r = reference[l].strip()
          h = hf[l].strip()
          json_lines.append({"reference":r,"hypothesis":h})
      with open(json_file, 'w') as ho:
          json.dump(json_lines, ho, indent=4)
  return system_names

The scores are written in MetricX/json/

In [ ]:
import numpy as np

%cd /content/metricx/

for lpair in translation_directions:
  system_names = make_metricX_json(lpair)
  jsonFile_dir = "/content/MetricX/json/"+lpair
  sys_metricx_scores = open("/content/MetricX/json/"+lpair+"/sys_scores.csv", "w")

  for n in system_names:
    jsonFile = jsonFile_dir+"/"+n+".jsonl"
    jsonFile_scores = jsonFile_dir+"/"+n+".scores.jsonl"
    !python predict.py \
      --tokenizer google/mt5-xl \
      --model_name_or_path google/metricx-23-xl-v2p0 \
      --max_input_length 1024 \
      --batch_size 1 \
      --input_file {jsonFile} \
      --output_file {jsonFile_scores}

    sscores = []
    print(jsonFile_scores)
    with open(jsonFile_scores) as seg_scores:
      for s in seg_scores:
        sscores.append(json.loads(s)['prediction'])
    sys_score = str(np.mean(sscores))
    sys_metricx_scores.write(n+","+sys_score+"\n")
  sys_metricx_scores.close()

#Evaluate with COMET (reference-free)

In [ ]:
!pip install --upgrade pip  # ensures that pip is current
!pip install unbabel-comet

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 529.7/529.7 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.4 which is incompatible.


Preparation of the dataset:
* **/!\ This code was used before we introduced the canary string in the files. You have to remove the first line of the sources and hypothesis files before running COMET. /!\**

In [ ]:

#COMET models are gated. You must have an HF account and provide your access token here:
from huggingface_hub import login
login(token="hf_#######")


#The translation directions to evaluate
translation_directions=["cs-uk", "en-cs", "en-de", "en-is", "en-es", "en-ja", "en-ru", "en-uk", "en-hi", "en-zh", "ja-zh"]
data_dir = "/content/wmt24-news-systems/txt/"

import os
def run_comet(tdirection):
  lpair = tdirection.split('-')
  src_lang = lpair[0]
  tgt_lang = lpair[1]
  source_file = data_dir+"/sources/{tdirection}.txt".format(tdirection=tdirection)
  hypothesis_directory = data_dir+"/system-outputs/"+tdirection
  hypothesis_files = [os.path.join(hypothesis_directory, f) for f in os.listdir(hypothesis_directory) if f.endswith(".txt") and os.path.isfile(os.path.join(hypothesis_directory, f))]
  hypothesis_files_all = " ".join(hypothesis_files)
  print("comet-score -s "+source_file+" -t "+hypothesis_files_all+" --model Unbabel/wmt23-cometkiwi-da-xl")
  os.system("comet-score -s "+source_file+" -t "+hypothesis_files_all+" --model Unbabel/wmt23-cometkiwi-da-xl > "+data_dir+"/comet."+tdirection+".scores")
  file_scores = open(data_dir+"/comet."+tdirection+".scores.csv", 'w')
  with open(data_dir+"/comet."+tdirection+".scores") as sc:
    for line in sc:
      if line.startswith("/content/wmt24-news-systems/txt/") and "Segment" not in line:
        score = line.strip().split()[-1]
        system_name = "".join(line.split('\t')[0].split('/')[-1].split('.')[:-1])
        file_scores.write(system_name+","+score+"\n")
  file_scores.close()


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
for lpair in translation_directions:
  run_comet(lpair)
